[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Niegil-Francis/astromer_notebooks/blob/main/Tutorial_finetuning_astromer.ipynb)

# Tutorial - Finetuning Astromer

This tutorial will go through the steps required to finetune [Astromer](https://www.stellardnn.org/astromer/pages/about_astromer.html).

##### *Install the library*

In [1]:
%%capture
!pip install test_model_hv

##### *Mount the drive containing the custom dataset*

In [2]:
from google.colab import drive
drive.mount('/content/drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##### *Import the necessary modules*

In [3]:
from core.astromer import ASTROMER 
from core.data import pretraining_data
from core.utils import get_folder_name
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import os
import warnings
import numpy as np
import pandas as pd
import argparse
import logging
import time
import os
from time import gmtime, strftime
warnings.filterwarnings("ignore")

##### *Load the Astromer model*

In [4]:
# Get model
astromer = ASTROMER(num_layers=2,
                        num_heads=4,
                        dff=128,
                        base=1000,
                        dropout=0.1,
                        maxlen=200,
                        no_train=True,
                        pretrained_weights="macho")
astromer.get_model().summary()

The saved weights already exists
Loading Base Model...
Weights loaded successfully!
Model: "ASTROMER"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 length (InputLayer)            [(None, 200)]        0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 200, 1)]     0           []                               
                                                                                                  
 times (InputLayer)             [(None, 200, 1)]     0           []                               
       

##### *Load the data*

In [5]:
os.makedirs('/content/Ogle', exist_ok = True)
# Make sure we don't overwrite a previous training
dir = get_folder_name("/content/Ogle/runs", prefix='')

# Creating directory
os.makedirs(dir, exist_ok=True)

# Loading data
train_batches = pretraining_data(os.path.join("/content/drive/Shareddrives/AstroMer/public/records/ogle_500", 'train'),
                                    256,
                                    max_obs=200,
                                    shuffle=True,
                                    msk_frac=0.5,
                                    rnd_frac=0.2,
                                    same_frac=0.2)
valid_batches = pretraining_data(os.path.join("/content/drive/Shareddrives/AstroMer/public/records/ogle_500", 'val'),
                                    256,
                                    max_obs=200,
                                    shuffle=False,
                                    msk_frac=0.5,
                                    rnd_frac=0.2,
                                    same_frac=0.2)

[INFO] Shuffling


##### *Finetune Astromer*

In [6]:
# Training ASTROMER
astromer.train(train_batches, valid_batches,
      patience=25,
      exp_path=dir,
      epochs=100,
      lr=0.001,
      verbose=1)

EPOCH 99 - ES COUNT: 2.0/25 train mse: 0.1031 - val mse: 0.0949: 100%|██████████| 100/100 [1:35:08<00:00, 57.09s/it]
